<a href="https://colab.research.google.com/github/AishwaryaVelumani/Interactive-Virtual-Art-Exhibition/blob/main/exhibition_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
#import necessary packages
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder 

In [42]:
from google.colab import files
uploaded= files.upload()

Saving data_for_exhibition_chatbot_monalisa.json to data_for_exhibition_chatbot_monalisa.json


In [43]:
#load training data for chatbot as json file
with open('data_for_exhibition_chatbot_monalisa.json') as file:
    data=json.load(file)

In [44]:
training_sentences=[]
training_labels=[]
labels=[]
responses=[]

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes=len(labels)
print(labels)

['greeting', 'goodbye', 'thanks', 'about', 'name', 'help', 'monalisa basic', 'monalisa dimensions']


In [45]:
lbl_encoder= LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels= lbl_encoder.transform(training_labels)
training_labels

array([2, 2, 2, 2, 2, 1, 1, 1, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 3, 3, 3, 3,
       3, 3, 3, 4, 4, 4, 5, 5])

In [46]:
vocab_size=1000
embedding_dim=16
max_len=20
oov_token="<OOV>"

tokenizer= Tokenizer(num_words=vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index= tokenizer.word_index
sequences= tokenizer.texts_to_sequences(training_sentences)
padded_sequences= pad_sequences(sequences,truncating='post',maxlen=max_len)

In [47]:
model= Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [48]:
epochs= 500
history= model.fit(padded_sequences,np.array(training_labels),epochs=epochs)

Epoch 1/500
1/1 [==============================] - 0s 480ms/step - loss: 2.0742 - accuracy: 0.2333
Epoch 2/500
1/1 [==============================] - 0s 15ms/step - loss: 2.0727 - accuracy: 0.2333
Epoch 3/500
1/1 [==============================] - 0s 15ms/step - loss: 2.0712 - accuracy: 0.2333
Epoch 4/500
1/1 [==============================] - 0s 15ms/step - loss: 2.0697 - accuracy: 0.2333
Epoch 5/500
1/1 [==============================] - 0s 14ms/step - loss: 2.0682 - accuracy: 0.2333
Epoch 6/500
1/1 [==============================] - 0s 18ms/step - loss: 2.0667 - accuracy: 0.2333
Epoch 7/500
1/1 [==============================] - 0s 17ms/step - loss: 2.0652 - accuracy: 0.2333
Epoch 8/500
1/1 [==============================] - 0s 14ms/step - loss: 2.0637 - accuracy: 0.2333
Epoch 9/500
1/1 [==============================] - 0s 16ms/step - loss: 2.0622 - accuracy: 0.2333
Epoch 10/500
1/1 [==============================] - 0s 13ms/step - loss: 2.0607 - accuracy: 0.2333
Epoch 11/500
1/1 [

In [49]:
#saving the training model
model.save("exhibition_chatbot")

import pickle

with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle','wb') as enc_file:
    pickle.dump(lbl_encoder,enc_file,protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: exhibition_chatbot/assets


In [50]:
import json
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
 

In [51]:
pip install colorama

In [52]:
import colorama
colorama.init()
from colorama import Fore,Style,Back

In [53]:
import random
import pickle

In [59]:
with open("data_for_exhibition_chatbot_monalisa.json") as file:
    data=json.load(file)

In [61]:
def chat():
    model= keras.models.load_model('exhibition_chatbot')

    with open('tokenizer.pickle','rb') as handle:
        tokenizer=pickle.load(handle)

    with open('label_encoder.pickle','rb') as enc:
        lbl_encoder= pickle.load(enc)

    max_len=20

    while True:
        print(Fore.LIGHTBLUE_EX + "User: "+ Style.RESET_ALL,end="")
        inp=input()
        if inp.lower()=='quit':
            break
        
        result= model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                                                         truncating='post',maxlen=max_len))
        tag= lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "Kala the Chatbot: "+ Style.RESET_ALL,np.random.choice(i['responses']))
print(Fore.YELLOW + "Start chatting with the bot: (type quit to stop)" + Style.RESET_ALL)
chat()            


Start chatting with the bot: (type quit to stop)
User: hi
Kala the Chatbot:  Hello
User: what is your name
Kala the Chatbot:  Just call me Kala
User: tell me about monalisa painting
Kala the Chatbot:  Monalisa is a 16th century painting painted in oil by Leonardo da Vinci during Renaissance in Florence, Italy
User: i want to know about monalisa dimensions
Kala the Chatbot:  Monalisa is 76 cm * 53 cm oil on poplar type painting
User: thanks bye
Kala the Chatbot:  See you later
User: quit
